In [ ]:
# 

In [56]:
# Setup
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from tqdm import trange

%matplotlib inline
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set random seed for reproducibility
seed = 1234
# cuDNN uses nondeterministic algorithms, set some options for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)

# read in dataset with date column parsed
df = pd.read_csv('cleanedWeatherAUS.csv',
    parse_dates=['Date'],
    index_col='Date')


y = df['RainTomorrow']
xs = df[df.columns.difference(['RainTomorrow'])]

# split dataset, 80% train 20% test
from sklearn.model_selection import train_test_split
train_data_x, test_data_x, train_data_y, test_data_y = train_test_split(xs, y, train_size=0.8)

In [80]:
import torch.utils.data as data_utils
train_data = data_utils.TensorDataset(torch.Tensor(np.array(train_data_x)), torch.Tensor(np.array(train_data_y)))
test_data = data_utils.TensorDataset(torch.Tensor(np.array(test_data_x)), torch.Tensor(np.array(test_data_y)))


# Initial transform (convert to PyTorch Tensor only)
transform = transforms.Compose([
    transforms.ToTensor(),
])


## Use the following lines to check the basic statistics of this dataset
# Calculate training data mean and standard deviation to apply normalization to data
# train_data.data are of type uint8 (range 0,255) so divide by 255.
# train_mean = train_data.data.double().mean() / 255.
# train_std = train_data.data.double().std() / 255.
# print(f'Train Data: Mean={train_mean}, Std={train_std}')

## Optional: Perform normalization of train and test data using calculated training mean and standard deviation
# This will convert data to be approximately standard normal
# transform = transforms.Compose([
#    transforms.ToTensor(),
#    transforms.Normalize((train_mean, ), (train_std, ))
# ])

train_data.transform = transform
test_data.transform = transform




batch_size = 64
torch.manual_seed(seed)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=True) 
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=True)

In [81]:
train_iter = iter(train_loader)
xs, labels = next(train_iter)



In [90]:
# input_size = 1 * 28 * 28  # input spatial dimension of images
input_size = 21
hidden_size = 64 #128         # width of hidden layer
output_size = 10          # number of output neurons


import math 

class NNClassifier(torch.nn.Module):
    
    def __init__(self):
        
        super().__init__()
        # self.flatten = torch.nn.Flatten(start_dim=1)

        # ------------------
        # Write your implementation here.
        # initialization -> define structure here, layers, weight is randomized

        # layer 1 input = vector
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        # ,512)

        # activation function - ReLU 
        self.act = torch.nn.ReLU()        
        
        # layer 2 output shape: (512, 10)?
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
            #512,10)
        
        # output
        self.log_softmax = torch.nn.Softmax(dim=1)

        # ------------------
    
    def forward(self, x):
        # x: Input image is of shape [batch_size, 1, 28, 28]
        
        # Need to flatten to [batch_size, 784] before feeding to fc1
        # x = self.flatten(x)

        # output after layer 1
        x = self.fc1(x)
        
        # apply activation function
        x = self.act(x)
        
        # output after layer 2
        x = self.fc2(x)
        
        # output after applying softmax
        return self.log_softmax(x)
        # ------------------

model = NNClassifier().to(DEVICE)

# sanity check
print(model)
# print([i in enumerate(model.parameters())])

NNClassifier(
  (fc1): Linear(in_features=21, out_features=64, bias=True)
  (act): ReLU()
  (fc2): Linear(in_features=64, out_features=10, bias=True)
  (log_softmax): Softmax(dim=1)
)


In [91]:
def train_one_epoch(train_loader, model, device, optimizer, log_interval, epoch):
    model.train()
    losses = []
    counter = []
    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    
    for i, (img, label) in enumerate(train_loader):
        img, label = img.to(device), label.to(device).long()
        
        # zero the gradients
        optimizer.zero_grad()
        # run forward on image data 
        outputs = model.forward(img)

        # compute loss
        loss = cross_entropy_loss(outputs, label)
        loss.backward()

        # adjust learning weights
        optimizer.step()
               
        # Record training loss every log_interval and keep counter of total training images seen
        if (i+1) % log_interval == 0:
            losses.append(loss.item())
            counter.append(
                (i * batch_size) + img.size(0) + epoch * len(train_loader.dataset))

    return losses, counter

In [92]:
def test_one_epoch(test_loader, model, device):
    model.eval()
    test_loss = 0
    num_correct = 0
    
    with torch.no_grad():
        for i, (features, label) in enumerate(test_loader):
            features, label = features.to(device), label.to(device)

            # ------------------
            # Write your implementation here.
            
            output = model.forward(features)
            pred = output.argmax(dim=1) 
            for iii in range(0, len(pred)):
                num_correct = num_correct + 1 if pred[iii] == label[iii] else num_correct + 0
                test_loss = test_loss + 0 if pred[iii] == label[iii] else test_loss + 1

            # ------------------
            
    test_loss /= len(test_loader.dataset)
    return test_loss, num_correct

In [93]:
# Hyperparameters
lr = 0.01
max_epochs=1
gamma = 0.95

# Recording data
log_interval = 100

# Instantiate optimizer (model was created in previous cell)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train_losses = []
train_counter = []
test_losses = []
test_correct = []
for epoch in trange(max_epochs, leave=True, desc='Epochs'):
    train_loss, counter = train_one_epoch(train_loader, model, DEVICE, optimizer, log_interval, epoch)
    test_loss, num_correct = test_one_epoch(test_loader, model, DEVICE)

    # Record results
    train_losses.extend(train_loss)
    train_counter.extend(counter)
    test_losses.append(test_loss)
    test_correct.append(num_correct)

print(f"Test accuracy: {test_correct[-1]/len(test_loader.dataset)}")

Epochs: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.75s/it]

Test accuracy: 0.0


In [ ]:
# 1. Draw training loss curve
fig = plt.figure(figsize=(12,8))
plt.plot(train_counter, train_losses, label='Train loss')
plt.plot([i * len(train_loader.dataset) for i in range(1, max_epochs + 1)], 
         test_losses, label='Test loss', marker='o')
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.title('Loss curve', fontsize=24)
plt.xlabel('Number of training examples seen', fontsize=16)
plt.ylabel('NLL', fontsize=16)
plt.legend(loc='upper right', fontsize=14)